In [11]:
from eodhd import APIClient
import pandas as pd
import datetime
import numpy as np
import calendar 
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
import csv
import requests
import json

In [12]:
api_key = "5cb671b0b4a790.35526238"
api = APIClient(api_key)

In [3]:
url = f'https://eodhd.com/api/fundamentals/GSPC.INDX?api_token={api_key}&fmt=json'
data = requests.get(url).json()
print(data)

{'General': {'Code': 'GSPC', 'Type': 'INDEX', 'Name': 'S&P 500 Index', 'Exchange': 'INDX', 'CurrencyCode': 'USD', 'CurrencyName': 'US Dollar', 'CurrencySymbol': '$', 'CountryName': 'USA', 'CountryISO': 'US', 'OpenFigi': None}, 'Components': {'0': {'Code': 'AIZ', 'Exchange': 'US', 'Name': 'Assurant Inc', 'Sector': 'Financial Services', 'Industry': 'Insurance - Specialty'}, '1': {'Code': 'MNST', 'Exchange': 'US', 'Name': 'Monster Beverage Corp', 'Sector': 'Consumer Defensive', 'Industry': 'Beverages - Non-Alcoholic'}, '2': {'Code': 'MTCH', 'Exchange': 'US', 'Name': 'Match Group Inc', 'Sector': 'Communication Services', 'Industry': 'Internet Content & Information'}, '3': {'Code': 'PGR', 'Exchange': 'US', 'Name': 'Progressive Corp', 'Sector': 'Financial Services', 'Industry': 'Insurance - Property & Casualty'}, '4': {'Code': 'CSX', 'Exchange': 'US', 'Name': 'CSX Corporation', 'Sector': 'Industrials', 'Industry': 'Railroads'}, '5': {'Code': 'ADP', 'Exchange': 'US', 'Name': 'Automatic Data P

In [4]:
component = data.get('Components',{})
file_path = 'component.json'

with open(file_path, 'w') as json_file:
    json.dump(component, json_file, indent=4)
print(f"JSON file created at: {file_path}")

JSON file created at: component.json


In [ ]:
df = pd.DataFrame.from_dict(component, orient='index')
csv_file_path = 'S&P500.csv'
df.to_csv(csv_file_path, index=False)

In [4]:
csv_file_path = 'S&P500.csv'
df = pd.read_csv(csv_file_path)

In [5]:
financial_list = []
for i in range(df.size):
    try:
        code = df.loc[i]['Code']
        financial = api.get_fundamentals_data(code)
        print(code)
        if financial:
            financial_list.append(financial)
    except Exception as e:
        print(f'error data for {code} : {e}')

AIZ
MNST
MTCH
PGR
CSX
ADP
ANSS
AFL
SO
REG
LMT
BMY
EIX
GWW
FAST
PHM
INTC
NEM
CTAS
ALL
CBOE
NDAQ
NRG
CSGP
GIS
CNC
MLM
QCOM
ACN
VMC
MRO
AVGO
AMT
PKG
CPB
FMC
RCL
WAT
PPL
FITB
DXCM
SNPS
HON
EW
APD
MCD
TDY
NTAP
KO
GS
AJG
NKE
KMB
ROL
HES
IT
CTSH
WEC
PG
SNA
CMA
BIO
IRM
EOG
FCX
TXN
AON
CHRW
FDS
GPN
CMG
TEL
MSCI
CF
TJX
WRB
DLTR
DRI
JPM
CB
KEY
GD
TGT
EQR
RJF
BLDR
SWKS
MS
AAL
RF
PANW
AEP
CE
AMAT
CCI
LULU
VTR
PPG
INTU
IPG
A
IFF
EL
CNP
AVY
UNP
MAS
PSX
GLW
CME
NCLH
SBUX
STE
CAG
PODD
CLX
UHS
ILMN
POOL
ODFL
MRNA
LUV
ZBRA
ROST
FFIV
HRL
TSCO
COP
CSCO
AEE
CL
DGX
AME
AOS
BIIB
BRO
MSI
ESS
HIG
CDNS
EQIX
EXR
CVX
HUM
BBY
PLD
OXY
T
KLAC
LLY
APH
PRU
EMN
IP
SPG
FICO
COF
CHD
PCAR
ORCL
FRT
PNW
RL
HCA
FIS
ZTS
CDW
URI
COST
AAPL
BA
DUK
BDX
PNR
K
PAYX
CAH
MKC
PEG
UDR
MA
GOOGL
DECK
HOLX
LVS
EPAM
BAC
MO
KMI
FE
HII
NSC
LEN
MOH
ENPH
ABT
WMB
TRV
VLO
CRM
SHW
WMT
MMM
NWS
KMX
AWK
DFS
ALGN
NVDA
LYB
EBAY
COO
XOM
FOX
UAL
DG
ETN
ULTA
MPWR
TSLA
TER
LOW
PAYC
ADM
NTRS
EMR
AMZN
ADSK
KIM
CAT
PM
PNC
JNJ
EXPD
TECH
C
MDLZ
TAP
AXP
ALB
AVB


In [6]:
json_file_path = 'financial_list.json'
with open(json_file_path, 'w') as json_file:
    json.dump(financial_list, json_file, indent=4) 

In [19]:
stock = financial_list[100]

In [20]:
url = f'https://eodhd.com/api/calendar/earnings?symbols=REG.US&from=2018-12-02&to=2018-12-06&api_token={api_key}&fmt=json'
Earning_data = requests.get(url)
print(Earning_data.text)

Forbidden. Please contact support@eodhistoricaldata.com


In [41]:
stock['Financials']['Income_Statement']['quarterly']['2024-04-30']['totalRevenue']

'1573000000.00'

In [36]:
def get_EPS(stock):
    dateList = []
    EarningList = []
    earningHistory = stock['Earnings']['History']
    cout = 0
    for key in earningHistory:
        EPS = earningHistory[key]['epsActual']
        if EPS is not None:
            cout = cout + 1
            dateList.append(key)
            EarningList.append(EPS)
            if cout == 28:
                return dateList, EarningList
    return dateList, EarningList

In [51]:
def get_Revenue(stock):
    dateList = []
    RevenueList = []
    RevenueHistory = stock['Financials']['Income_Statement']['quarterly']
    cout = 0
    for key in RevenueHistory:
        Revenue = RevenueHistory[key]['totalRevenue']
        if Revenue is not None:
            Revenue = float(Revenue)
            cout = cout + 1
            dateList.append(key)
            RevenueList.append(Revenue)
            if cout == 28:
                return dateList, RevenueList

In [59]:
def get_Asset(stock):
    dateList = []
    outcomeList = []
    RevenueHistory = stock['Financials']['Balance_Sheet']['quarterly']
    cout = 0
    for key in RevenueHistory:
        Revenue = RevenueHistory[key]['totalAssets']
        if Revenue is not None:
            Revenue = float(Revenue)
            cout = cout + 1
            dateList.append(key)
            outcomeList.append([key, Revenue])
            if cout == 22:
                return outcomeList

In [60]:
def get_Liabilities(stock):
    dateList = []
    outcomeList = []
    LiabHistory = stock['Financials']['Balance_Sheet']['quarterly']
    cout = 0
    for key in LiabHistory:
        Liab = LiabHistory[key]['totalLiab']
        if Liab is not None:
            Liab = float(Liab)
            cout = cout + 1
            dateList.append(key)
            outcomeList.append([key, Liab])
            if cout == 22:
                return outcomeList

In [73]:
def get_Equity(stock):
    dateList = []
    outcomeList = []
    EquityHistory = stock['Financials']['Balance_Sheet']['quarterly']
    cout = 0
    for key in EquityHistory:
        Equity = EquityHistory[key]['totalStockholderEquity']
        if Equity is not None:
            Equity = float(Equity)
            cout = cout + 1
            dateList.append(key)
            outcomeList.append([key, Equity])
            if cout == 22:
                return outcomeList

In [55]:
def get_Cost_Revenue(stock):
    dateList = []
    CostList = []
    CostHistory = stock['Financials']['Income_Statement']['quarterly']
    cout = 0
    for key in CostHistory:
        Cost = CostHistory[key]['costOfRevenue']
        if Cost is not None:
            Cost = float(Cost)
            cout = cout + 1
            dateList.append(key)
            CostList.append(Cost)
            if cout == 28:
                return dateList, CostList

In [78]:
def get_debt(stock):
    debt1 = None
    debt2 = None
    debt = None
    cout = 0
    debtlist = []
    report = stock['Financials']['Balance_Sheet']['quarterly']
    for key in report:
        debt1 = report[key].get('longTermDebtTotal')
        debt2 = report[key].get('longTermDebt')
        if debt1 and debt2:
            debt = float(max(debt1, debt2))
        if debt1:
            debt = float(debt1)
        if debt2:
            debt = float(debt2)
        if debt:
            debtlist.append([key,debt])
            cout += 1
        if cout == 28:
            return debtlist

In [42]:
def get_list(dateList, dataList):
    cout = 0
    currentSum = 0
    finalList = []
    listsize = 0
    outputList = []
    while cout < 4:
        currentSum += dataList[cout]
        cout += 1

    while listsize < 22:
        finalList.append(currentSum)
        currentSum += dataList[cout]
        currentSum -= dataList[cout-4]
        cout += 1
        listsize += 1
    for i in range(listsize):
        outputList.append([dateList[i], finalList[i]])
    return outputList

In [48]:
dateList, dataList = get_EPS(stock)
print(get_list(dateList, dataList))
dateList_2, RevenueList = get_Revenue(stock)
print(get_list(dateList_2, RevenueList))


[['2024-04-30', 5.319999999999999], ['2024-01-31', 5.37], ['2023-10-31', 5.45], ['2023-07-31', 5.6000000000000005], ['2023-04-30', 5.510000000000001], ['2023-01-31', 5.370000000000001], ['2022-10-31', 5.210000000000001], ['2022-07-31', 4.890000000000001], ['2022-04-30', 4.65], ['2022-01-31', 4.49], ['2021-10-31', 4.340000000000001], ['2021-07-31', 4.11], ['2021-04-30', 3.7900000000000005], ['2021-01-31', 3.5300000000000002], ['2020-10-31', 3.28], ['2020-07-31', 3.19], ['2020-04-30', 3.17], ['2020-01-31', 3.17], ['2019-10-31', 3.1199999999999997], ['2019-07-31', 3.0399999999999996], ['2019-04-30', 2.9499999999999993], ['2019-01-31', 2.8899999999999992]]
[['2024-04-30', 6591000000.0], ['2024-01-31', 6735000000.0], ['2023-10-31', 6833000000.0], ['2023-07-31', 6994000000.0], ['2023-04-30', 7040000000.0], ['2023-01-31', 6930000000.0], ['2022-10-31', 6848000000.0], ['2022-07-31', 6659000000.0], ['2022-04-30', 6527000000.0], ['2022-01-31', 6445000000.0], ['2021-10-31', 6319000000.0], ['2021-0

In [61]:
AssetList = get_Asset(stock)
print(AssetList)
LiabList = get_Liabilities(stock)
print(LiabList)

[['2024-04-30', 10856000000.0], ['2024-01-31', 10948000000.0], ['2023-10-31', 10763000000.0], ['2023-07-31', 10675000000.0], ['2023-04-30', 10792000000.0], ['2023-01-31', 10919000000.0], ['2022-10-31', 10516000000.0], ['2022-07-31', 10484000000.0], ['2022-04-30', 10455000000.0], ['2022-01-31', 10327000000.0], ['2021-10-31', 10705000000.0], ['2021-07-31', 10491000000.0], ['2021-04-30', 10398000000.0], ['2021-01-31', 9674000000.0], ['2020-10-31', 9627000000.0], ['2020-07-31', 9546000000.0], ['2020-04-30', 9455000000.0], ['2020-01-31', 9501000000.0], ['2019-10-31', 9452000000.0], ['2019-07-31', 8625000000.0], ['2019-04-30', 9022000000.0], ['2019-01-31', 8952000000.0]]
[['2024-04-30', 4642000000.0], ['2024-01-31', 4760000000.0], ['2023-10-31', 4918000000.0], ['2023-07-31', 5117000000.0], ['2023-04-30', 5011000000.0], ['2023-01-31', 5310000000.0], ['2022-10-31', 5227000000.0], ['2022-07-31', 5393000000.0], ['2022-04-30', 5333000000.0], ['2022-01-31', 5173000000.0], ['2021-10-31', 5316000000

In [65]:
dateList, CostList = get_Cost_Revenue(stock)
print(get_list(dateList, CostList))
DebtList = get_debt(stock)
print(DebtList)

[['2024-04-30', 3249000000.0], ['2024-01-31', 3325000000.0], ['2023-10-31', 3368000000.0], ['2023-07-31', 3432000000.0], ['2023-04-30', 3197000000.0], ['2023-01-31', 3150000000.0], ['2022-10-31', 3126000000.0], ['2022-07-31', 3049000000.0], ['2022-04-30', 3004000000.0], ['2022-01-31', 2966000000.0], ['2021-10-31', 2912000000.0], ['2021-07-31', 2847000000.0], ['2021-04-30', 2705000000.0], ['2021-01-31', 2578000000.0], ['2020-10-31', 2502000000.0], ['2020-07-31', 2437000000.0], ['2020-04-30', 2427000000.0], ['2020-01-31', 2415000000.0], ['2019-10-31', 2358000000.0], ['2019-07-31', 2313000000.0], ['2019-04-30', 2273000000.0], ['2019-01-31', 2266000000.0]]
[['2024-04-30', 2136000000.0], ['2024-01-31', 2555000000.0], ['2023-10-31', 2735000000.0], ['2023-07-31', 2734000000.0], ['2023-04-30', 2733000000.0], ['2023-01-31', 2733000000.0], ['2022-10-31', 2733000000.0], ['2022-07-31', 2732000000.0], ['2022-04-30', 2730000000.0], ['2022-01-31', 2730000000.0], ['2021-10-31', 2729000000.0], ['2021-0

In [74]:
EquityList = get_Equity(stock)
print(EquityList)
print(stock['Financials']['Balance_Sheet']['quarterly']['2024-04-30']['totalStockholderEquity'])
print(stock['Financials']['Balance_Sheet']['quarterly']['2024-04-30']['liabilitiesAndStockholdersEquity'])
print(stock['Financials']['Balance_Sheet']['quarterly']['2024-04-30']['totalLiab'])

[['2024-04-30', 6214000000.0], ['2024-01-31', 6188000000.0], ['2023-10-31', 5845000000.0], ['2023-07-31', 5558000000.0], ['2023-04-30', 5781000000.0], ['2023-01-31', 5609000000.0], ['2022-10-31', 5289000000.0], ['2022-07-31', 5091000000.0], ['2022-04-30', 5122000000.0], ['2022-01-31', 5154000000.0], ['2021-10-31', 5389000000.0], ['2021-07-31', 4946000000.0], ['2021-04-30', 4810000000.0], ['2021-01-31', 4804000000.0], ['2020-10-31', 4873000000.0], ['2020-07-31', 4981000000.0], ['2020-04-30', 4768000000.0], ['2020-01-31', 4848000000.0], ['2019-10-31', 4748000000.0], ['2019-07-31', 4747000000.0], ['2019-04-30', 5125000000.0], ['2019-01-31', 5036000000.0]]
6214000000.00
10856000000.00
4642000000.00


In [83]:
codelist = []
EPSlist = []
RevenueList = []
AssetList = []
Costlist = []
Liabilities = []
DebtList = []
EquityList = []
for stock in financial_list:
    code = stock['General']['Code']
    codelist.append(code)
    try:
        EPSlist.append([code, get_list(*get_EPS(stock))])
    except Exception as e:
        print(f"{code}EPS Error: {e}")
        continue
    try:
        RevenueList.append([code, get_list(*get_Revenue(stock))])
    except Exception as e:
        print(f"{code}Revenue Error: {e}")
        continue
    try:
        AssetList.append([code, get_Asset(stock)])
    except Exception as e:
        print(f"{code}Asset Error: {e}")
        continue
    try:
        CostList.append([code, get_list(*get_Cost_Revenue(stock))])
    except Exception as e:
        print(f"{code}Cost Error: {e}")
        continue
    try:
        Liabilities.append([code, get_Liabilities(stock)])
    except Exception as e:
        print(f"{code}Liab Error: {e}")
        continue
    try:
        DebtList.append([code, get_debt(stock)])
    except Exception as e:
        print(f"{code}Debt Error: {e}")
        continue
    try:
        EquityList.append([code, get_Equity(stock)])
    except Exception as e:
        print(f"{code}Equity Error: {e}")
        continue
   

MRNAEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
FOXAEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
VICIEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
DAYEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
UBEREPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
TFCCost Error: __main__.get_list() argument after * must be an iterable, not NoneType
CTVAEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
AMCREPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
CARREPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
OTISEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
ABNBEPS Error: __main__.get_list() argument after * must be an iterable, not NoneType
CEGEPS Error: __main__.get_list() argument after * must

In [85]:
data = {
    "EPSlist": EPSlist,
    "AssetList": AssetList,
    "Costlist": Costlist,
    "Liabilities": Liabilities,
    "DebtList": DebtList,
    "EquityList": EquityList
}

with open('ListofData.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)
